In [1]:
import os
import math
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset
from torch.profiler import profile, record_function, ProfilerActivity
import matplotlib.pyplot as plt
import psutil
import time
from mpl_toolkits.axes_grid1 import ImageGrid
import csv
import gc
import itertools
import yaml
import seaborn as sns
import scipy
from mytools import *


In [3]:
## Enabling garbage collector to save RAM
gc.enable()

## User configuration
device = 'cpu'
dataset_path = "./data/"
input_name = "quat_10000_32^3.npy"
gt_name = "sigma_10000_32^3.npy"
model_path="./trained_models/"
model_name = "bestmodel_selfsup_extension_xx_inner_depth_150_n_resblocks_4.pt"
outfolder='./post_proc_results/'
n_train_val_samples = 6000 ## ALREADY USED FOR TRAINING AND VALIDATION
n_test_samples = 1000 ## UNTOUCHED TEST DATA
plot_dat_file_crops = True
plot_crop = 0
crop_positions = [0, 11, 21]

## Creating analysis output folder
os.system("mkdir -p " + outfolder)

## Loading datasets
quat_dataset = np.load(dataset_path + input_name)
sigma_fft_dataset = np.load(dataset_path + gt_name)

In [4]:
## Separating the dataset into separate tensors
quat_test = (
    torch.from_numpy(quat_dataset[n_train_val_samples : n_train_val_samples + n_test_samples])
    .to(gettensorprecision(precision))
    .to(device)
)
sigma_fft_test = (
    torch.from_numpy(
        sigma_fft_dataset[n_train_val_samples : n_train_val_samples + n_test_samples]
    )
    .to(gettensorprecision(precision))
    .to(device)
)

## Manually freeing some RAM
del quat_dataset
del sigma_fft_dataset
torch.cuda.empty_cache()

In [ ]:
## Loading model in evaluation mode (we don't need to calculate gradients)
trained_model = torch.load(model_path + model_name, map_location=torch.device(device))
trained_model = trained_model.module.to(device)
trained_model.eval()

## Initializing arrays
concat_fft_test = np.array([])
concat_nn_test = np.array([])
concat_vm_fft_test = np.array([])
concat_vm_nn_test = np.array([])
concat_grain_avg_gt = np.array([])
concat_grain_avg_pred = np.array([])
concat_grain_avg_gt_vm = np.array([])
concat_grain_avg_pred_vm = np.array([])
concat_grainsizes = np.array([])

In [ ]:
## Starting clock
t0 = time.time()

## Analysis main loop
for sample in range(n_test_samples):

    concat_fft_test = np.concatenate(
        (concat_fft_test, sigma_fft_test[sample, 0, :, :, :].cpu().detach().numpy().flatten())
    )

    concat_vm_fft_test = np.concatenate(
        (concat_vm_fft_test, vm_vector(sigma_fft_test[sample].cpu()).flatten())
    )

    ## FOR MEASURING RAM AND TIME
    # with torch.no_grad():
    #     with profile(activities=[ProfilerActivity.CPU],profile_memory=True, record_shapes=True) as prof:
    #         sig_nn_test  = trained_model(quat_test[sample:sample+1])
    # print('self_cpu_memory_usage \n')
    # print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))
    # print('cpu_memory_usage \n')
    # print(prof.key_averages().table(sort_by="cpu_memory_usage", row_limit=10))

    with torch.no_grad():
        sig_nn_test = trained_model(quat_test[sample : sample + 1])

    sig_nn_vm = vm_matrix(sig_nn_test.cpu())

    sig_fft_vm = vm_vector(sigma_fft_test[sample].cpu())

    concat_vm_nn_test = np.concatenate(
        (concat_vm_nn_test, vm_matrix(sig_nn_test.cpu()).flatten())
    )

    concat_nn_test = np.concatenate(
        (concat_nn_test, sig_nn_test[0, 0, 0, :, :, :].cpu().detach().numpy().flatten())
    )

    grain_avg_gt_sample, grain_avg_pred_sample, grainsizes_sample = (
        performance_scatterplot(
            quat_test[sample : sample + 1].to("cpu"),
            sigma_fft_test[sample : sample + 1, 0:1, :, :, :].to("cpu"),
            sig_nn_test[0:1, 0:1, 0, :, :, :].cpu(),
            outfolder,
            "sigxx_",
        )
    )

    grain_avg_gt_sample_vm, grain_avg_pred_sample_vm, grainsizes_sample_vm = (
        performance_scatterplot(
            quat_test[sample : sample + 1].cpu(),
            torch.Tensor(np.expand_dims(np.expand_dims(sig_fft_vm, axis=0), axis=0)).cpu(),
            torch.Tensor(np.expand_dims(np.expand_dims(sig_nn_vm, axis=0), axis=0)).cpu(),
            outfolder,
            "sig_eq_",
        )
    )

    concat_grain_avg_gt = np.concatenate((concat_grain_avg_gt, grain_avg_gt_sample))
    concat_grain_avg_pred = np.concatenate(
        (concat_grain_avg_pred, grain_avg_pred_sample)
    )

    concat_grain_avg_gt_vm = np.concatenate(
        (concat_grain_avg_gt_vm, grain_avg_gt_sample_vm)
    )
    concat_grain_avg_pred_vm = np.concatenate(
        (concat_grain_avg_pred_vm, grain_avg_pred_sample_vm)
    )

    concat_grainsizes = np.concatenate((concat_grainsizes, grainsizes_sample))

    if plot_dat_file_crops == True and sample == plot_crop:
        data_fft = sigma_fft_test[plot_crop, 0, :, :, :].cpu().detach().numpy()
        data_nn = sig_nn_test[0, 0, 0, :, :, :].cpu().detach().numpy()
        data_fft_vm = sig_fft_vm[:, :, :].cpu().detach().numpy()
        data_nn_vm = np.squeeze(sig_nn_vm[:, :, :].cpu().detach().numpy())
        data_error_abs = abs(data_fft - data_nn)
        data_error_rel = data_error_abs / data_fft
        data_error_abs_vm = abs(data_fft_vm - data_nn_vm)
        data_error_rel_vm = data_error_abs_vm / data_fft_vm
        n = data_nn.shape[0]

        for x in crop_positions:
            f = open(outfolder + "sig_nn_xx_position_" + str(x) + ".dat", "w")
            g = open(outfolder + "sig_fft_xx_position_" + str(x) + ".dat", "w")
            a = open(outfolder + "err_abs_sig_xx_position_" + str(x) + ".dat", "w")
            r = open(outfolder + "err_rel_sig_xx_position_" + str(x) + ".dat", "w")
            ff = open(outfolder + "sig_nn_vm_position_" + str(x) + ".dat", "w")
            gg = open(outfolder + "sig_fft_vm_position_" + str(x) + ".dat", "w")
            aa = open(outfolder + "err_abs_sig_vm_position_" + str(x) + ".dat", "w")
            rr = open(outfolder + "err_rel_sig_vm_position_" + str(x) + ".dat", "w")

            # f.write('y,z,data \n')
            for z in range(n):
                for y in range(n):
                    # f.write(str(y)+' '+str(z)+' '+str(data_nn[x,y,z])+'\n')
                    # g.write(str(y)+' '+str(z)+' '+str(data_fft[x,y,z])+'\n')
                    f.write(str(y) + " " + str(z) + " " + str(data_nn[x, y, z]) + "\n")
                    g.write(str(y) + " " + str(z) + " " + str(data_fft[x, y, z]) + "\n")
                    a.write(
                        str(y)
                        + " "
                        + str(z)
                        + " "
                        + str(data_error_abs[x, y, z])
                        + "\n"
                    )
                    r.write(
                        str(y)
                        + " "
                        + str(z)
                        + " "
                        + str(100 * data_error_rel[x, y, z])
                        + "\n"
                    )
                    ff.write(
                        str(y) + " " + str(z) + " " + str(data_nn_vm[x, y, z]) + "\n"
                    )
                    gg.write(
                        str(y) + " " + str(z) + " " + str(data_fft_vm[x, y, z]) + "\n"
                    )
                    aa.write(
                        str(y)
                        + " "
                        + str(z)
                        + " "
                        + str(data_error_abs_vm[x, y, z])
                        + "\n"
                    )
                    rr.write(
                        str(y)
                        + " "
                        + str(z)
                        + " "
                        + str(100 * data_error_rel_vm[x, y, z])
                        + "\n"
                    )
                f.write("\n")
                g.write("\n")
                a.write("\n")
                r.write("\n")
                ff.write("\n")
                gg.write("\n")
                aa.write("\n")
                rr.write("\n")
            f.close()
            g.close()
            a.close()
            r.close()
            ff.close()
            gg.close()
            aa.close()
            rr.close()

## Stop clock and save training time
t1 = time.time()
total_time = t1 - t0
f = open(outfolder + "inference_time.txt", "w")
f.write(
    "The inference lasted  "
    + convert_time(total_time)
    + " for "
    + str(n_test_samples)
    + " samples."
)
f.close()

In [ ]:
## CORRELATION DIAGRAM sigma_xx (voxelwise)
plt.figure()
plt.ticklabel_format(axis="both", style="sci", scilimits=(0, 0))
plt.scatter(concat_fft_test, concat_nn_test, alpha=0.05)
plt.plot(
    concat_fft_test,
    concat_fft_test,
    color="red",
    linestyle="--",
    linewidth=2,
    label="Identity line",
)
plt.title("Voxelwise values")
plt.xlabel("FFT ground truth (MPa)")
plt.ylabel("NN prediction for sigma_xx (MPa)")


## CORRELATION DIAGRAM sigma_xx (grain avgs)
plt.figure()
plt.ticklabel_format(axis="both", style="sci", scilimits=(0, 0))
plt.scatter(concat_grain_avg_gt, concat_grain_avg_pred, alpha=0.2)
plt.plot(
    concat_grain_avg_gt,
    concat_grain_avg_gt,
    color="red",
    linestyle="-",
    linewidth=2,
    label="Identity line",
)
plt.legend()
plt.axis("equal")
plt.title("Grainwise averages")
plt.xlabel("FFT ground truth (MPa)")
plt.ylabel("NN prediction for sigma_xx (MPa)")

## CORRELATION DIAGRAM sigma_eq (voxelwise)
sig_fft_vm_vector = sig_fft_vm.flatten()
sig_nn_vm_vector = sig_nn_vm.flatten()
plt.figure()
plt.ticklabel_format(axis="both", style="sci", scilimits=(0, 0))
plt.scatter(sig_fft_vm_vector, sig_nn_vm_vector, alpha=0.05)
plt.plot(
    sig_fft_vm_vector,
    sig_fft_vm_vector,
    color="red",
    linestyle="--",
    linewidth=2,
    label="Identity line",
)
plt.title("Voxelwise values")
plt.xlabel("FFT ground truth (MPa)")
plt.ylabel("NN prediction for sigma_eq (MPa)")

## CORRELATION DIAGRAM sigma_eq (grain avgs)
plt.figure()
plt.ticklabel_format(axis="both", style="sci", scilimits=(0, 0))
plt.scatter(concat_grain_avg_gt_vm, concat_grain_avg_pred_vm, alpha=0.2)
plt.plot(
    concat_grain_avg_gt_vm,
    concat_grain_avg_gt_vm,
    color="red",
    linestyle="--",
    linewidth=2,
    label="Identity line",
)
plt.title("Grainwise averages")
plt.xlabel("FFT ground truth (MPa)")
plt.ylabel("NN prediction for sigma_eq (MPa)")


## RELATIVE ERROR CDFs
concat_rel_err = abs(concat_fft_test - concat_nn_test) / concat_fft_test
concat_rel_err_vm = abs(concat_vm_fft_test - concat_vm_nn_test) / concat_vm_fft_test
avg_rel_err = concat_rel_err.mean()
avg_rel_err_vm = concat_rel_err_vm.mean()

f = open(outfolder + "avg_relative_error_" + str(n_test_samples) + "_samples.txt", "w")
f.write("The average relative error is  " + str(avg_rel_err))
f.close()

f = open(outfolder + "avg_relative_error_vm" + str(n_test_samples) + "_samples.txt", "w")
f.write("The average relative error for Von Mises stress is  " + str(avg_rel_err_vm))
f.close()

np.savetxt(
    outfolder + "fft_flat_" + str(n_test_samples) + "_samples.dat",
    concat_fft_test,
    fmt="%d",
)
np.savetxt(
    outfolder + "fft_flat_vm_" + str(n_test_samples) + "_samples.dat",
    concat_vm_fft_test,
    fmt="%d",
)
np.savetxt(
    outfolder + "nn_flat_" + str(n_test_samples) + "_samples.dat",
    concat_nn_test,
    fmt="%d",
)
np.savetxt(
    outfolder + "nn_flat_vm_" + str(n_test_samples) + "_samples.dat",
    concat_vm_nn_test,
    fmt="%d",
)
np.savetxt(
    outfolder + "grain_avg_gt_flat_" + str(n_test_samples) + "_samples.dat",
    concat_grain_avg_gt,
    fmt="%d",
)
np.savetxt(
    outfolder + "grain_avg_pred_flat_" + str(n_test_samples) + "_samples.dat",
    concat_grain_avg_pred,
    fmt="%d",
)
np.savetxt(
    outfolder + "grain_avg_gt_flat_vm_" + str(n_test_samples) + "_samples.dat",
    concat_grain_avg_gt_vm,
    fmt="%d",
)
np.savetxt(
    outfolder + "grain_avg_pred_flat_vm_" + str(n_test_samples) + "_samples.dat",
    concat_grain_avg_pred_vm,
    fmt="%d",
)
np.savetxt(
    outfolder + "grainsizes_flat_" + str(n_test_samples) + "_samples.dat",
    concat_grainsizes,
    fmt="%d",
)
np.savetxt(
    outfolder + "rel_err_flat_" + str(n_test_samples) + "_samples.dat",
    concat_rel_err,
    fmt="%.5f",
)
np.savetxt(
    outfolder + "rel_err_flat_vm_" + str(n_test_samples) + "_samples.dat",
    concat_rel_err_vm,
    fmt="%.5f",
)


err_xx = error_data("rel_err_flat_" + str(n_test_samples) + "_samples.dat", outfolder)
err_vm = error_data("rel_err_flat_vm_" + str(n_test_samples) + "_samples.dat", outfolder)

err_xx.load()
err_vm.load()

err_xx.pdf()
err_vm.pdf()

err_xx.cdf()
err_vm.cdf()

np.savetxt(outfolder+"data_err_xx.dat", err_xx.data, fmt="%f")
np.savetxt(outfolder+"x_err_xx.dat", err_xx.x, fmt="%f")
np.savetxt(outfolder+"pdf_err_xx.dat", err_xx.pdf, fmt="%f")
np.savetxt(outfolder+"cdf_err_xx.dat", err_xx.cdf, fmt="%f")

np.savetxt(outfolder+"data_err_vm.dat", err_vm.data, fmt="%f")
np.savetxt(outfolder+"x_err_vm.dat", err_vm.x, fmt="%f")
np.savetxt(outfolder+"pdf_err_vm.dat", err_vm.pdf, fmt="%f")
np.savetxt(outfolder+"cdf_err_vm.dat", err_vm.cdf, fmt="%f")
